
# Hello world

In this unit you will learn how to use Python to implement the first ever program
that *every* programmer starts with. This also serves as an example for the master
notebook format.

```
# All markdown cells are searched for triple-backtick blocks. Within a triple quoted block,
# a match on /^# ASSIGNMENT METADATA/ will trigger handling this as an assignment-level metadata
# block. The rest of the metadata is parsed as YAML and may be used e.g. for setting
# default settings for autograder isolation (memory limit etc).
# The assignment_id is useful to identify which assignment a submission pertains to.
# ASSIGNMENT METADATA
assignment_id: "HelloWorld"
```

## Introduction

Here is the traditional first programming exercise, called "Hello world".
The task is to print the message: "Hello, world".

Here are a few examples to get you started. Run the following cells and see how
you can print a message. To run a cell, click with mouse inside a cell, then
press Ctrl+Enter to execute it. If you want to execute a few cells sequentially,
then press Shift+Enter instead, and the focus will be automatically moved
to the next cell as soon as one cell finishes execution.

In [1]:
print("hello")

hello


In [2]:
print("bye bye")

bye bye


In [3]:
print("hey", "you")

hey you


In [4]:
print("one")
print("two")

one
two


## MASTER ONLY. A few helper functions and IPython magics

In [5]:
# MASTER ONLY
import unittest

# TODO(salikh): Move this into a shared library and make that library installable via pip.
class SummaryTestResult(unittest.TextTestResult):
    """A small extension of TextTestResult that also collects a map of test statuses.
    
    result.results is a map from test name (string) to boolean: True(passed) or False(failed or error)"""
    
    separator1 = '=' * 70
    separator2 = '-' * 70
    
    def __init__(self, stream, descriptions, verbosity):
        super(unittest.TextTestResult, self).__init__(stream, descriptions, verbosity)
        # A map of test name to True(passed) or False(failed or error)
        self.results = {}
        # Copied from TextTestResult.
        self.stream = stream
        self.showAll = verbosity > 1
        self.dots = verbosity == 1
        self.descriptions = descriptions

    def testName(self, test):
        """A helper function to format the test as a human-readable string.
        
        The format is TestClassName.test_method. This is similar
        to TextTestResult.getDescription(test), but uses different format.
        getDescription: 'test_one (__main__.HelloTest)'
        testName: 'HelloTest.test_one'
        """
        return unittest.util.strclass(test.__class__).replace('__main__.', '') + '.' + test._testMethodName
        
    def getDescription(self, test):
        doc_first_line = test.shortDescription()
        if self.descriptions and doc_first_line:
            return '\n'.join((str(test), doc_first_line))
        else:
            return str(test)

    def startTest(self, test):
        super(unittest.TextTestResult, self).startTest(test)
        if self.showAll:
            self.stream.write(self.getDescription(test))
            self.stream.write(" ... ")
            self.stream.flush()

    def addSuccess(self, test):
        super(unittest.TextTestResult, self).addSuccess(test)
        if self.showAll:
            self.stream.writeln("ok")
        elif self.dots:
            self.stream.write('.')
            self.stream.flush()
        self.results[self.testName(test)] = True

    def addError(self, test, err):
        super(unittest.TextTestResult, self).addError(test, err)
        if self.showAll:
            self.stream.writeln("ERROR")
        elif self.dots:
            self.stream.write('E')
            self.stream.flush()
        self.results[self.testName(test)] = False

    def addFailure(self, test, err):
        super(unittest.TextTestResult, self).addFailure(test, err)
        if self.showAll:
            self.stream.writeln("FAIL")
        elif self.dots:
            self.stream.write('F')
            self.stream.flush()
        self.results[self.testName(test)] = False

    def addSkip(self, test, reason):
        super(unittest.TextTestResult, self).addSkip(test, reason)
        if self.showAll:
            self.stream.writeln("skipped {0!r}".format(reason))
        elif self.dots:
            self.stream.write("s")
            self.stream.flush()

    def addExpectedFailure(self, test, err):
        super(unittest.TextTestResult, self).addExpectedFailure(test, err)
        if self.showAll:
            self.stream.writeln("expected failure")
        elif self.dots:
            self.stream.write("x")
            self.stream.flush()

    def addUnexpectedSuccess(self, test):
        super(unittest.TextTestResult, self).addUnexpectedSuccess(test)
        if self.showAll:
            self.stream.writeln("unexpected success")
        elif self.dots:
            self.stream.write("u")
            self.stream.flush()

    def printErrors(self):
        if self.dots or self.showAll:
            self.stream.writeln()
        self.printErrorList('ERROR', self.errors)
        self.printErrorList('FAIL', self.failures)

    def printErrorList(self, flavour, errors):
        for test, err in errors:
            self.stream.writeln(self.separator1)
            self.stream.writeln("%s: %s" % (flavour,self.getDescription(test)))
            self.stream.writeln(self.separator2)
            self.stream.writeln("%s" % err)


In [6]:
# MASTER ONLY. TODO(salikh): Extract the magics code into a pip library.

from types import SimpleNamespace
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

import re

@register_cell_magic
def submission(line, cell):
    """Registers a submission_source and submission, if the code can run.
    
    This magic is useful for auto-testing (testing autograder unit tests on incorrect inputs)"""
    
    # Copy the source into submission_source.source
    globals()['submission_source'] = SimpleNamespace(source=cell)

    env = {}
    try:
        exec(cell, globals(), env)
    except Exception as e:
        # Ignore execution errors -- just print them.
        print('Exception while executing submission:\n', e)
        # If the code cannot be executed, leave the submission empty.
        globals()['submission'] = None
        return None
    # Copy the modifications into submission object.
    globals()['submission'] = SimpleNamespace(**env)

@register_cell_magic
def solution(line, cell):
    """Registers solution and evaluates it.
    
    Also removes the PROMPT block and %%solution from the solution source.
    
    The difference from %%submission is that the solution is copied into the top context,
    making it possible to refer to the functions and variables in subsequent notebook cells."""
    
    # Cut out PROMPT
    cell = re.sub('(?ms)[ \t]*""" # BEGIN PROMPT.*""" # END PROMPT[ \t]*\n?', '', cell)
    # Cut out BEGIN/END SOLUTION markers
    cell = re.sub('(?ms)[ \t]*# (BEGIN|END) SOLUTION[ \t]*\n?', '', cell)

    # Copy the source into submission_source.source
    globals()['submission_source'] = SimpleNamespace(source=cell)
    
    env = {}
    # Note: if solution throws exception, this breaks the execution. Solution must be correct!
    exec(cell, globals(), env)
    # Copy the modifications into submission object.
    globals()['submission'] = SimpleNamespace(**env)
    # Copy the modifications into globals().
    for k in env:
        globals()[k] = env[k]

import sys
import io
        
@register_line_magic
def autotest(line):
    """Run the unit tests inline
    
    Returns the result object. result.results is a summary dictionary of outcomes.
    result.errors 
    """
    
    suite = unittest.TestLoader().loadTestsFromTestCase(eval(line))
    errors = io.StringIO()
    result = unittest.TextTestRunner(verbosity=4, stream=errors, resultclass=SummaryTestResult).run(suite) 
    return result, errors.getvalue()

# Remove the names from the global namespace.
del autotest, submission, solution

In [7]:
# MASTER ONLY. TODO(salikh): Extract the magics code into a pip library.

from jinja2 import Template
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

from IPython.core.display import HTML

@register_cell_magic
def template(line, cell):
    """Registers a template for report generation.
    
    Use {{results['TestClassName.test_method']}} to extract specific outcomes and be prepared
    for the keys to be absent in the results map, if the test could not be run at all (e.g. because
    of syntax error in the submission).
    """
    name = line
    if line == '':
        name = 'report_template'
    # Define a Jinja2 template based on cell contents.
    globals()[name] = Template(cell)

@register_cell_magic
def report(line, cell):
    """Renders the named template.
    
    Syntax:
      %%report results_var
      template_name
    """
    var_name = line
    template_name = cell
    template = eval(template_name, globals())
    results = eval(var_name, globals())
    highlighted_source = highlight(submission_source.source, PythonLexer(), HtmlFormatter())
    # Render the template giving the specified variable as 'results',
    # and render the result as inlined HTML in cell output. 'source' is
    # the prerendered source code.
    return HTML(template.render(results=results, source=highlighted_source))

# Remove the names from the global namespace.
del template, report

## Exercise 1: printing greeting

```
# The markdown cell with triple-backtick block matching /^# EXERCISE METADATA/ is an exercise-level
# metadata. The next block is assumed to be the solution block, and will get annotated with
# the exercise_id.
# EXERCISE METADATA
exercise_id: "hello1"
```

Now it is your turn. Please create a program in the next cell that would print a message "Hello, world":

In [8]:
%%solution
""" # BEGIN PROMPT
# ... put your program here
""" # END PROMPT
# BEGIN SOLUTION
print("Hello, world")
# END SOLUTION

Hello, world


In [9]:
# This will not be included in the student notebook because of BEGIN UNITTEST marker below.

# The test below assumes that the solution above was written into two files:
# - submission.py with the solution code
# - submission_source.py which defines a single variable source whose value holds the source
#   code of the submission.
#
# and then imported with
#
#   import submission_source
#   import submission
#
# In the Jupyter notebook, this setup is performed by %%solution and %%submission magics.

# BEGIN UNITTEST
# The unit tests main part is contained between "BEGIN UNITTEST" and "END UNITTEST". It will be copied
# verbatim into the autograder directory, with an addition of 'import submission' and 'import submission_source'
import unittest
#import submission_source

import sys
import io
from contextlib import contextmanager
from io import StringIO

# TODO(salikh): Move the helper code into a library.
@contextmanager
def capture_output():
    """Captures the stdout and stderr into StringIO objects."""
    capture_out, capture_err = StringIO(), StringIO()
    save_out, save_err = sys.stdout, sys.stderr
    try:
        sys.stdout, sys.stderr = capture_out, capture_err
        yield sys.stdout, sys.stderr
    finally:
        sys.stdout, sys.stderr = save_out, save_err

class HelloOutputTest(unittest.TestCase):
    def test_output(self):
        with capture_output() as (out, err):
            exec(submission_source.source)
        self.assertEqual(err.getvalue(), "")
        self.assertEqual(out.getvalue(), "Hello, world\n")
        
    def test_not_empty(self):
        with capture_output() as (out, err):
            exec(submission_source.source)
        self.assertNotEqual(out.getvalue(), "")
        
    def test_has_hello(self):
        with capture_output() as (out, err):
            exec(submission_source.source)
        self.assertTrue("Hello" in out.getvalue())

# END UNITTEST

# The parts after END UNITTEST are executed in the notebook environment, but not copied
# to the autograder scripts or to student notebooks.

result, log = %autotest HelloOutputTest
# Optional. Useful for debugging.
print(log)
assert(result.results['HelloOutputTest.test_output'] == True)

test_has_hello (__main__.HelloOutputTest) ... ok
test_not_empty (__main__.HelloOutputTest) ... ok
test_output (__main__.HelloOutputTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK



In [10]:
%%template HelloOutputTest_template
<h2 style='color: #387;'>Your submission</h2>
<pre style='background: #F0F0F0; padding: 3pt; margin: 4pt; border: 1pt solid #DDD; border-radius: 3pt;'
{{ source }}
</pre>
<h2 style='color: #387;'>Results</h2>
{% if not results['HelloOutputTest.test_not_empty'] %}
Your snippet does not produce any output. Please add a print statement.
{% elif not results['HelloOutputTest.test_has_hello'] %}
The output of your code does not include "Hello" string. Please add it.
{% elif not results['HelloOutputTest.test_output'] %}
The output is incorrect. Please make sure it is exactly "Hello, world".
{% else %}
Your code looks okay.
{% endif %}

In [11]:
%%report result.results
HelloOutputTest_template

## Exercise 2: returning greeting as value

```
# EXERCISE METADATA
exercise_id: "hello2"
```

Please create a function that given a name string returns a string with a greeting,
For example, for input `"world"` it should return `"Hello, world"`.

In [12]:
%%solution
def hello(name):
    """ # BEGIN PROMPT
    # Please put your solution here:
    # return ...
    pass
    """ # END PROMPT
    # BEGIN SOLUTION
    return "Hello, " + name
    # END SOLUTION

In [13]:
# TEST
assert(hello("world") == "Hello, world")

A code cell marked with `"# TEST"` will be converted into a unit test for the solution. It will also be preserved in the student version of the notebook.

##### MASTER ONLY

`# MASTER ONLY` marker works in text cells too.

In [14]:
# The part before "BEGIN UNITTEST" -- preamble -- sets up the environment so that 'submission.hello'
# is a function that we need to test. In the autograder worker environment, the preamble will be
# replaced with 'import submission' with an assumption that the student's solution will be written
# to the file 'submission.py'. The submission source will be available in submission_source.py
# that will define a single variable named 'source'.

# The unit tests main part is contained between "BEGIN UNITTEST" and "END UNITTEST". It will be copied
# verbatim into the autograder directory, with an addition of 'import submission' and 'import submission_source'

# BEGIN UNITTEST
#import submission

import unittest
import ast

class HelloTest(unittest.TestCase):
    
    def test_hello(self):
        self.assertEqual(submission.hello("one"), "Hello, one")
        
    def test_includes_arg(self):
        self.assertTrue("xyz123" in submission.hello("xyz123"))
        
    def test_includes_hello(self):
        self.assertTrue("Hello" in submission.hello("xyz123"))

# END UNITTEST

result, log = %autotest HelloTest
# Optional.
print(log)
print(result.results)
assert result.results['HelloTest.test_hello'] == True

test_hello (__main__.HelloTest) ... ok
test_includes_arg (__main__.HelloTest) ... ok
test_includes_hello (__main__.HelloTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK

{'HelloTest.test_includes_arg': True, 'HelloTest.test_hello': True, 'HelloTest.test_includes_hello': True}


In [15]:
%%template HelloTest_template
<h2 style='color: #387;'>Your submission</h2>
<pre style='background: #F0F0F0; padding: 3pt; margin: 4pt; border: 1pt solid #DDD; border-radius: 3pt;'
{{ source }}
</pre>
<h2 style='color: #387;'>Results</h2>
{% if not results['HelloTest.py'] %}
Your code does not compile
{% elif not results['HelloTest.test_includes_hello'] %}
The response from your function does not include "Hello" string. Please check if you have included it.
{% elif not results['HelloTest.test_includes_arg'] %}
The response from your function does not include the person name, which as given as the function argument.
{% elif not results['HelloTest.test_hello'] %}
Something is wrong.
{% else %}
Your code looks okay.
{% endif %}

In [16]:
%%report result.results
HelloTest_template

In [17]:
# BEGIN UNITTEST
#import submission_source
import unittest
import ast

class HelloSyntaxTest(unittest.TestCase):
    def test_compiles(self):
        # Expect success or exception.
        ast.parse(submission_source.source)
        
# END UNITTEST        

result, log = %autotest HelloSyntaxTest
# Optional.
print(log)
print(result.results)
assert result.results['HelloSyntaxTest.test_compiles'] == True

test_compiles (__main__.HelloSyntaxTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK

{'HelloSyntaxTest.test_compiles': True}


In [18]:
%%template HelloSyntaxTest_template
<style type='text/css'>
.k { color: purple; }
.c1 { color: green; }
.s2 { color: brown; }
</style>
<h2 style='color: #387;'>Your submission</h2>
<pre style='background: #F0F0F0; padding: 3pt; margin: 4pt; border: 1pt solid #DDD; border-radius: 3pt;'
{{ source }}
</pre>
<h2 style='color: #087;'>Results</h2>
{% if not results['HelloSyntaxTest.test_compiles']: %}
Do you even compile, bro?
{% else %}
Syntax looks fine.
{% endif %}

In [19]:
%%report result.results
HelloSyntaxTest_template

In [20]:
%%submission
# The tests below test that the unit test suite above produces expected outcomes from
def hello(name):
    return "Bye, " + name

In [21]:
result1, log = %autotest HelloTest
#print(log)
assert result1.results['HelloTest.test_hello'] == False
assert result1.results['HelloTest.test_includes_hello'] == False
assert result1.results['HelloTest.test_includes_arg'] == True

In [22]:
%%report result1.results
HelloTest_template

In [23]:
result2, log = %autotest HelloSyntaxTest
assert result2.results['HelloSyntaxTest.test_compiles'] == True
results = {**result1.results, **result2.results}

In [24]:
%%report result2.results
HelloSyntaxTest_template

In [25]:
%%report result2.results
HelloTest_template

In [26]:
%%submission
def wrong_hello(name):
    return "Bye, " + name

In [27]:
result, log = %autotest HelloSyntaxTest
print(log)
print(result.results)

test_compiles (__main__.HelloSyntaxTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK

{'HelloSyntaxTest.test_compiles': True}


In [28]:
%%submission
# This submission has a syntax error. Note that invalid code inside %%submission cell
# does not break the notebook execution.
def syntax_error(name:
    return name

Exception while executing submission:
 invalid syntax (<string>, line 4)


In [29]:
# MASTER ONLY
# In case of syntax error in submission, it is initiliazed to None value.
assert submission == None
# TODO(salikh): Fix the behavior, since in the autograder 'import submission' will fail,
# producing empty outcome vector rather than a bunch of ERROR outcomes.

In [30]:
# %autotest expects the submission to have been set with %%submission cell magic previously.
result3, log = %autotest HelloSyntaxTest
print(log)
print(result3.results)
assert(result3.results['HelloSyntaxTest.test_compiles'] == False)

test_compiles (__main__.HelloSyntaxTest) ... ERROR

ERROR: test_compiles (__main__.HelloSyntaxTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-17-991943cb22a2>", line 9, in test_compiles
    ast.parse(submission_source.source)
  File "/usr/lib/python3.5/ast.py", line 35, in parse
    return compile(source, filename, mode, PyCF_ONLY_AST)
  File "<unknown>", line 4
    return name
         ^
SyntaxError: invalid syntax

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)

{'HelloSyntaxTest.test_compiles': False}


In [31]:
%%report result3.results
HelloSyntaxTest_template